In [1]:
import time
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
import os
import re
import pandas as pd
import numpy as np
from IPython.display import Audio
import seaborn as sns

import helpers # this is where the main training/decoding functions are, modified from teh original HIVAE main.py

#import warnings 
#warnings.filterwarnings('ignore') ########## NOTE: comment out for testing in case it's hiding problems

def set_settings(opts,nepochs=500,modload=False,save=True): # note: modload doesnt do anything right now, hardcoded in helpers.py
    'replace setting template placeholders with file info'
    inputf=re.sub('.csv','',opts['files'].iloc[0])
    missf=inputf+'_missing.csv'
    typef=inputf+'_types.csv'
    
    template = '--epochs NEPOCHS --model_name model_HIVAE_inputDropout --restore MODLOAD \
        --data_file data_python/INPUT_FILE.csv --types_file data_python/TYPES_FILE \
         --batch_size NBATCH --save NEPFILL --save_file SAVE_FILE\
        --dim_latent_s SDIM --dim_latent_z 1 --dim_latent_y YDIM \
         --miss_percentage_train 0 --miss_percentage_test 0 \
        --true_miss_file data_python/MISS_FILE --learning_rate LRATE'
    
    # replace placeholders in template
    settings = re.sub('INPUT_FILE',inputf,template)
    settings = re.sub('NBATCH',str(opts['nbatch'].iloc[0]),settings)
    settings = re.sub('NEPOCHS',str(nepochs),settings)
    settings = re.sub('NEPFILL',str(nepochs-1),settings) if save else re.sub('NEPFILL',str(nepochs*2),settings)
    settings = re.sub('YDIM',str(opts['ydims'].iloc[0]),settings)
    settings = re.sub('SDIM',str(opts['sdims'].iloc[0]),settings)
    settings = re.sub('MISS_FILE',missf,settings) #if not 'medhist' in inputf else re.sub('--true_miss_file data_python/MISS_FILE','',settings)
    settings = re.sub('TYPES_FILE',typef,settings)
    settings = re.sub('SAVE_FILE',inputf,settings)
    settings = re.sub('LRATE',str(opts['lrates'].iloc[0]),settings)
    settings = re.sub('MODLOAD','1',settings) if modload else re.sub('MODLOAD','0',settings)
    
    return settings

In [2]:
sample_size=178
# get file list
files=[i for i in os.listdir('data_python/') if not '_type' in i and not '_missing' in i and i not in '.DS_Store']
files.sort()
print(files)

['ARObjectFinding.csv', 'ARPlaceAndFindTelemetryVariance.csv', 'ARScreenButtonPresses.csv', 'BITDOTMotorInstructionReadingTimeRatios.csv', 'MMSE_Attention_Concentration.csv', 'MMSE_Language.csv', 'MMSE_Memory_Recall.csv', 'MMSE_Orientation.csv', 'MMSE_Working_Memory_Registration.csv', 'MotorDrawingFeatures.csv', 'MotorTappingFeatures.csv', 'MotorTestDurations.csv']


# General settings

In [3]:
best_hyper = pd.read_csv('best_hyper_ALTOIDA_processed.csv')
best_hyper

,lrates,nbatch,wdecay,ydims,files,loss,sdims
0,0.001,16,0.001,1,ARObjectFinding.csv,24.957258,1
1,0.001,16,0.010,1,ARPlaceAndFindTelemetryVariance.csv,-290.294244,1
2,0.010,16,0.000,1,ARScreenButtonPresses.csv,22.558093,1
3,0.010,16,0.000,1,BITDOTMotorInstructionReadingTimeRatios.csv,4.106932,1
4,0.010,32,0.010,1,MMSE_Attention_Concentration.csv,0.910142,4
5,0.010,32,0.010,1,MMSE_Language.csv,0.613249,2
6,0.001,16,0.001,1,MMSE_Memory_Recall.csv,1.080130,3
7,0.010,32,0.000,1,MMSE_Orientation.csv,0.688555,4
8,0.010,32,0.001,1,MMSE_Working_Memory_Registration.csv,0.058547,1
9,0.010,16,0.001,1,MotorDrawingFeatures.csv,68.877012,1


# VP decoding

Run after bnet.R

In [4]:
os.getcwd()

'/Users/meemansasood/Documents/Documents_IT/VAMBN_Version5_mmseEncoded_final/data/HI-VAE'

In [5]:
meta = pd.read_csv('metaenc.csv')
meta

,scode_ARObjectFinding,SUBJID,zcode_ARObjectFinding,scode_ARPlaceAndFindTelemetryVariance,zcode_ARPlaceAndFindTelemetryVariance,scode_ARScreenButtonPresses,zcode_ARScreenButtonPresses,scode_BITDOTMotorInstructionReadingTimeRatios,zcode_BITDOTMotorInstructionReadingTimeRatios,scode_MMSE_Attention_Concentration,...,scode_MMSE_Orientation,zcode_MMSE_Orientation,scode_MMSE_Working_Memory_Registration,zcode_MMSE_Working_Memory_Registration,scode_MotorDrawingFeatures,zcode_MotorDrawingFeatures,scode_MotorTappingFeatures,zcode_MotorTappingFeatures,scode_MotorTestDurations,zcode_MotorTestDurations
0,0,009-20_001_1,-1.042224,0,0.935868,0,0.585778,0,0.680546,3,...,3,-0.916167,0,1.523856,0,0.988264,0,-3.996608,0,1.106418
1,0,009-20_002_1,0.128795,0,2.264648,0,-0.106377,0,-0.517770,2,...,1,10.482780,0,1.523856,0,0.364599,0,0.016033,0,0.654828
2,0,009-20_003_1,1.216998,0,1.235406,0,0.070480,0,0.689196,2,...,1,10.482780,0,1.523856,0,0.361916,0,0.629590,0,-1.276689
3,0,009-20_004_1,0.270090,0,2.717478,0,-0.446318,0,-0.118558,2,...,3,-0.916167,0,1.523856,0,-0.080454,0,1.088204,0,-0.612896
4,0,009-20_005_1,-0.627163,0,2.300178,0,0.156115,0,0.362346,2,...,0,3.068466,0,0.961632,0,-0.087803,0,0.182084,0,-0.426074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0,005_012_2,2.632748,0,3.557675,0,-0.353706,0,-1.246386,3,...,1,10.482780,0,1.523856,0,-0.401649,0,-1.877998,0,0.281586
174,0,005_012_3,2.256792,0,3.426479,0,-1.651626,0,-1.246386,3,...,1,10.482780,0,1.523856,0,-0.401649,0,-1.877998,0,0.281586
175,0,005_015_1,2.004861,0,3.820880,0,-0.348762,0,-1.170140,3,...,0,3.068466,0,1.523856,0,-0.028437,0,-0.521731,0,-0.310235
176,0,005_015_2,2.142821,0,2.720714,0,-1.004784,0,-1.170140,3,...,0,3.068466,0,1.523856,0,-0.028437,0,-0.521731,0,-0.310235


In [7]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/VAMBN_Version5_mmseEncoded_final/data/data_out/main_VirtualPPts_10.csv')
VPcodes

,zcode_ARObjectFinding,zcode_ARPlaceAndFindTelemetryVariance,zcode_ARScreenButtonPresses,zcode_BITDOTMotorInstructionReadingTimeRatios,scode_MMSE_Attention_Concentration,zcode_MMSE_Attention_Concentration,scode_MMSE_Language,zcode_MMSE_Language,scode_MMSE_Memory_Recall,zcode_MMSE_Memory_Recall,...,SA_yearsOfEducation_VIS1,SA_DX_VIS1,SA_Amyloid_VIS1,AUX_SA_DX_VIS1,AUX_MMSE_Attention_Concentration_VIS1,AUX_MMSE_Language_VIS1,AUX_MMSE_Memory_Recall_VIS1,AUX_MMSE_Orientation_VIS1,AUX_MMSE_Working_Memory_Registration_VIS1,visitmiss_VIS1
0,-0.177119,2.638057,-0.306168,-0.012441,2,7.248290,1,0.191511,0,4.042812,...,19.117882,1,0,0,0,1,0,0,1,0
1,-0.433077,1.649321,0.316748,-0.794143,3,0.730296,0,-7.366124,2,-1.141633,...,6.037999,4,1,0,0,0,0,0,0,0
2,2.442221,2.834292,-1.162448,0.498965,2,7.248290,0,-7.277397,0,3.969586,...,11.703218,1,0,0,0,0,0,1,1,0
3,0.668548,2.540274,0.389526,-0.310567,3,-0.218969,0,-7.400230,2,-0.364174,...,7.121089,2,1,0,0,0,0,0,1,0
4,0.081226,1.502781,0.648992,0.687618,2,7.248290,0,-6.988706,0,3.875481,...,16.148971,3,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1775,-0.676143,1.987050,1.323556,-0.292694,2,7.248290,1,-0.337520,2,-0.127732,...,13.197279,1,0,0,0,1,0,1,0,0
1776,1.626966,2.747440,-0.368695,-0.015869,3,-0.498820,0,-7.237735,2,-1.287739,...,16.501075,2,1,0,0,0,1,0,1,0
1777,0.865261,1.443331,0.128631,-1.815757,3,1.302375,0,-7.245074,2,-1.651284,...,16.973470,2,1,0,1,0,0,0,1,0
1778,2.902551,3.148697,-1.683484,0.446603,2,7.248290,0,-7.024231,2,-0.043890,...,14.109391,0,0,0,0,0,1,1,0,0


In [8]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/VAMBN_Version5_mmseEncoded_final/data/data_out/main_VirtualPPts_10.csv')
dfs=list()
virt=list()
decs=list()
sample_size=178
n=sample_size
#decoded = []
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
    #dec=helpers.dec_network(settings,zcodes,scodes,VP=True)
    decs = pd.DataFrame()
    for i in range(int(len(VPcodes)/n)):
        dec=helpers.dec_network(settings,zcodes[i*n:(i*n+n)],scodes[i*n:(i*n+n)],VP=True);
        dec = pd.DataFrame(dec)
        #decs = np.concatenate((decs, dec), axis = 0)
        decs = decs.append(dec)
    #decs = pd.DataFrame(decs)
    subj=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_subj.csv')['x']
    names=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_cols.csv')['x']
    dat=pd.DataFrame(decs)
    dat.columns=names
    dat['SUBJID']=list(range(len(dat)))
    #dat =dat.assign(SUBJID=list(range(len(dat[1]))))
    virt.append(decs)
    dfs.append(dat)


decoded=helpers.merge_dat(dfs)
decoded.to_csv('decodedVP_10.csv',index=False)

[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders




/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Encoder...
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


[*] Defining Decoder...
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Th

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectFinding/ARObjectFinding.ckpt
Model restored: ./Saved_Networks/ARObjectFinding/ARObjectFinding.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/ARObjectFinding_vpmiss.csv
memememe
[[ 1.          1.         62.859      ...  7.          0.
   0.        ]
 [ 0.          1.         11.799      ...  3.          0.
   0.        ]
 [ 0.          0.         15.60733333 ...  1.          0.
   0.        ]
 ...
 [ 0.          0.          6.79266667 ...  2.          0.
   0.        ]
 [ 0.          0.         10.06866667 ...  0.          0.
   0.        ]
 [ 0.          0.         13.09933333 ...  0.          0.
   0.        ]]
memememe
[[ 1.          1.         62.859      ...  7.          0.
   0.        ]
 [ 0.          1.         11.799      ...  3.          0.
   0.        ]
 [ 0.          0.         15.60733333 ...  1.          0.
   0.        ]


[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectFinding/ARObjectFinding.ckpt
Model restored: ./Saved_Networks/ARObjectFinding/ARObjectFinding.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/ARObjectFinding_vpmiss.csv
memememe
[[ 1.          1.         62.859      ...  7.          0.
   0.        ]
 [ 0.          1.         11.799      ...  3.          0.
   0.        ]
 [ 0.          0.         15.60733333 ...  1.          0.
   0.        ]
 ...
 [ 0.          0.          6.79266667 ...  2.          0.
   0.        ]
 [ 0.          0.         10.06866667 ...  0.          0.
   0.        ]
 [ 0.          0.         13.09933333 ...  0.          0.
   0.        ]]
memememe
[[ 1.          1.         62.859      ...  7.          0.
   0.        ]
 [ 0.          1.         11.799      ...  3.          0.
   0.        ]
 [ 0.          0.         15.60733333 ...  1.          0.
   0.        ]


...

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
Model restored: ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/MotorTestDurations_vpmiss.csv
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.58

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
Model restored: ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/MotorTestDurations_vpmiss.csv
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.58

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
Model restored: ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/MotorTestDurations_vpmiss.csv
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.58

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
Model restored: ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/MotorTestDurations_vpmiss.csv
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.58

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
Model restored: ./Saved_Networks/MotorTestDurations/MotorTestDurations.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/MotorTestDurations_vpmiss.csv
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.585 13.03 ]
 [ 5.368  2.151  1.917 ...  9.91   2.206 19.312]
 [ 1.993  1.207  1.085 ...  9.327  2.605 13.017]
 ...
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]
 [ 4.667  2.045  1.922 ...  9.533  2.154 12.816]]
memememe
[[ 5.152  2.183  0.209 ...  7.851  3.58

Get Loglikelihoods for R plot!

In [10]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/VAMBN_Version5_mmseEncoded_final/data/data_out/main_VirtualPPts_10.csv')
sample_size=178
n=sample_size
dfs=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
    logliks = pd.DataFrame()
    for i in range(int(len(VPcodes)/n)):   
        loglik=helpers.dec_network_loglik(settings,zcodes,scodes,VP=True)
        loglik=np.nanmean(np.array(loglik).T,axis=1)
        loglik = pd.DataFrame(loglik)
        #decs = np.concatenate((decs, dec), axis = 0)
        logliks = logliks.append(loglik)
        print(logliks)
    subj=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_subj.csv')['x']
    dat=pd.DataFrame(logliks)
    dat.columns=[f]
    dat['SUBJID']=subj
    dfs.append(dat)

decoded=helpers.merge_dat(dfs)
decoded.to_csv('virtual_logliks_10.csv',index=False)

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=178, data_file='data_python/ARObjectFinding.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.001, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ARObjectFinding', true_miss_file='data_python/ARObjectFinding_missing.csv', types_file='data_python/ARObjectFinding_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectFinding/ARObjectFinding.ckpt
Model restored: ./Saved_Networks/ARObjectFinding/ARObjectFinding.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/ARObjectFinding_vpmiss.csv


ValueError: Cannot feed value of shape (1780, 1) for Tensor 'Placeholder_31:0', which has shape '(178, 1)'